In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
np.set_printoptions(suppress=True)
pd.options.mode.chained_assignment = None
data = pd.read_csv('all_stocks_5yr.csv')
df =data[data['Name']=='SO']
X_df = df[['close', 'low', 'high', 'open']]
def standardize(X):
    sum = 0
    for each in X:
        sum += each
    avg = sum / len(X)

    X_std = []
    for i in range(len(X)):
        normal = (avg - X[i])
        X_std.append(normal / np.linalg.norm(X[i]) )  
    return np.array(X_std)
    
def measure1(X, it, vec, no_class):
    for i in range((it % len(X)), len(X)):
        p_max = []
        for j in range(no_class):
            p_max.append(np.dot(vec[j], X[i])) 
        max_index = (np.where(p_max == np.amax(p_max)))

        modify = (max_index[0][0])
        return modify 

def measure2(X, it, vec, no_class):
    for i in range((it % len(X)), len(X)):
        p_min = []
        for j in range(no_class):
            p_min.append(np.dot(vec[j], X[i])) 
        min_index = (np.where(p_min == np.amin(p_min)))

        modify = (min_index[0][0])
        return modify

def measure3(X, it, vec, no_class):
    for i in range((it % len(X)), len(X)):
        p_min = []
        for j in range(no_class):
            temp = 0
            for k in range(len(vec[0])):
                temp += abs(vec[j][k] - X[i][k])
            p_min.append(np.dot(vec[j], X[i])) 
        min_index = (np.where(p_min == np.amin(p_min)))

        modify = (min_index[0][0])
        return modify

def Kohonen(X, no_class, alpha, no_it, measure, mod_alpha, C, C2, std):
    if std:
        X_norm = standardize(X)
    else:
        X_norm = np.array(X)
    
    random_generate = np.random.RandomState(61232)

    vec = []

    for i in range(no_class):
        temp_vec = random_generate.normal(loc=0.0, scale=0.01, size=len(X[0]))
        vec.append(temp_vec/np.linalg.norm(temp_vec))
    representing_vec = np.array(vec)

    fin_alpha = alpha

    for iterations in range(no_it):
        if measure == 1:
            modify = measure1(X_norm, iterations, representing_vec, no_class)
        elif measure == 2:
            modify = measure2(X_norm, iterations, representing_vec, no_class)
        elif measure == 3:
            modify = measure3(X_norm, iterations, representing_vec, no_class)
        
        representing_vec[modify] = representing_vec[modify] + fin_alpha * (X_norm[iterations % len(X_norm)] - representing_vec[modify])
        representing_vec[modify] = representing_vec[modify] / np.linalg.norm(representing_vec[modify])
    return representing_vec

def prepare_data(data, index, limit):
    X,Y = [],[]
    for i in range(len(data) - 1 - limit):
        x = np.array(data[i:i+limit])
        xa = x.reshape(x.shape[0]*x.shape[1])
        ya = np.array(data[(i+limit):(i+limit+1)][index])
        X.append(xa)
        Y.append(ya)
    return np.array(X), np.array(Y)


def calculate_radius(no_class, centers, std_data):
    max = 0
    for i in range(len(std_data)):
        for j in range(len(std_data)):
            msr = np.linalg.norm(std_data[i] - std_data[j])
            if msr > max:
                max = msr
    msr = max
    radius = msr

    return radius


def activation_func(data, radius):
    fi=np.exp(-np.square(data)/radius)
    return fi


def FI_matrix(centers, data, radius, no_class):
    FI=[]
    for i in range(len(data)):
        row=[]
        for j in range(no_class):
            abs_val = np.linalg.norm(data[i] - centers[j])
            element = activation_func(abs_val, radius)
            row.append(element)
        FI.append(row)
    return np.array(FI)

def weights(FI, expected_vals):
    FI_T= np.transpose(FI)
    
    weights_vec = np.dot(np.dot(np.linalg.pinv(np.dot(FI_T, FI)), FI_T), expected_vals)
    return weights_vec

def network_activation(weights, data, radius, centers):
    network_activation = []
    for i in range(len(data)):
        sum=0
        for j in range(no_class):
            abs_val = np.linalg.norm(data[i] - centers[j])
            element = activation_func(abs_val, radius)
            sum = sum + (weights_vec[j] * element)
        network_activation.append(sum)
    return network_activation

In [2]:
alpha = 0.7
no_it = 50
measure = 3
mod_alpha = 2
C = 1.5
C2 = 1.7
std = True

no_class = 20
limit = 5

data, expected_vals = prepare_data(X_df, 'close', limit)
centers = Kohonen(data, no_class, alpha, no_it, measure, mod_alpha, C, C2, std)
std_data = standardize(data)
radius = calculate_radius(no_class, centers, std_data)
print(type(centers))
print(type(std_data))
print(type(radius))
FI = FI_matrix(centers, std_data, radius, no_class)
weights_vec = weights(FI, expected_vals)    
activation_vec = network_activation(weights_vec, std_data, radius, centers)



<class 'NoneType'>
<class 'numpy.ndarray'>
<class 'numpy.float64'>


TypeError: 'NoneType' object is not subscriptable